### 验证 Whoosh的工呢过

#### 载入必要的依赖包

In [8]:
import sys,os
sys.path.append("../")
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser

from jieba.analyse.analyzer import ChineseAnalyzer

analyzer = ChineseAnalyzer()

#### 定义 SSC 所需要的 Schema

In [9]:
corpus_schema = Schema(id=ID(stored=True), corpus_id=ID(stored=True), field=ID(stored=True), content=TEXT(stored=True, analyzer=analyzer))


if not os.path.exists("corpus_index_dir"):
    os.mkdir("corpus_index_dir")

#### 添加文档

In [10]:
ix = create_in('corpus_index_dir', corpus_schema, 'bot_1')
writer = ix.writer()
writer.add_document(id='123', corpus_id='1', field='faq', content='你们公司在哪里')
writer.add_document(id='124', corpus_id='1', field='faq', content='你们公司在几号线')
writer.commit()

In [11]:
ix2 = create_in('corpus_index_dir', corpus_schema, 'bot_2')
writer2 = ix2.writer()
writer2.add_document(id='123', corpus_id='1', field='faq', content='你们公司在哪里')
writer2.add_document(id='124', corpus_id='1', field='faq', content='你们公司在几号线')
writer2.commit()

#### 提交变化

#### 重新加载文档

In [16]:
import whoosh.index as index

ix = index.open_dir('corpus_index_dir', indexname='bot_1')
parser = QueryParser('content', schema=ix.schema)

with ix.searcher() as s:
    q = parser.parse('公司')
    results = s.search(q)
    for hit in results:
        # print(hit.highlights("content")
        print(hit.get('field'))
    print("==" * 10)

faq


### 测试删除 document

In [17]:

writer1 = ix.writer()
# writer1.delete_by_term('id', '123')
writer1.update_document(id='124', corpus_id='1', field='faq', content='你们公司在几号线!!')

writer1.commit()

In [20]:
ix = index.open_dir('corpus_index_dir', indexname='bot_1')
parser = QueryParser('content', schema=ix.schema)

with ix.searcher() as s:
    q = parser.parse('公司')
    results = s.search(q)
    for hit in results:
        # print(hit.highlights("content")
        print(hit.get('field'), hit.get('content'))
    print("==" * 10)

faq 你们公司在几号线
faq 你们公司在几号线!!
